# Hands-on RL with Ray’s RLlib
## A beginner’s tutorial for working with multi-agent environments, models, and algorithms

### Overview
“Hands-on RL with Ray’s RLlib” is a beginners tutorial for working with reinforcement learning (RL) environments, models, and algorithms using Ray’s RLlib library. RLlib offers high scalability, a large list of algos to choose from (offline, model-based, model-free, etc..), support for TensorFlow and PyTorch, and a unified API for a variety of applications. This tutorial includes a brief introduction to provide an overview of concepts (e.g. why RL) before proceeding to RLlib (multi- and single-agent) environments, neural network models, hyperparameter tuning, debugging, student exercises, Q/A, and more. All code will be provided as .py files in a GitHub repo.

### Intended Audience
* Python programmers who want to get started with reinforcement learning and RLlib.

### Prerequisites
* Some Python programming experience.
* Some familiarity with machine learning.
* *Helpful, but not required:* Experience in reinforcement learning and Ray.
* *Helpful, but not required:* Experience with TensorFlow or PyTorch.

### Requirements/Dependencies

Install conda (https://www.anaconda.com/products/individual)

Then ...

#### Quick `conda` setup instructions (Mac and Linux):
```
$ conda create -n rllib python=3.8
$ conda activate rllib
$ pip install ray[rllib]
$ pip install [tensorflow|torch]  # <- either one works!
$ pip install jupyter-labs
```

#### Quick `conda` setup instructions (Win10):
```
$ conda create -n rllib python=3.8
$ conda activate rllib
$ pip install ray[rllib]
$ pip install [tensorflow|torch]  # <- either one works!
$ pip install jupyter-labs
$ conda install pywin32
```

Also, for Win10 Atari support, we have to install atari_py from a different source (gym does not support Atari envs on Windows).

```
$ pip install git+https://github.com/Kojoley/atari-py.git
```

### Opening these tutorial files:
```
$ git clone https://github.com/sven1977/rllib_tutorials
$ cd rllib_tutorials
$ jupyter-lab
```

### Key Takeaways
* What is reinforcement learning and why RLlib?
* Core concepts of RLlib: Environments, Trainers, Policies, and Models.
* How to configure, hyperparameter-tune, and parallelize RLlib.
* RLlib debugging best practices.

### Tutorial Outline
1. RL and RLlib in a nutshell.
1. Defining an RL-solvable problem: Our first environment.
1. Exercise No.1 (env loop)
1. Picking an algorithm and training our first RLlib Trainer.
1. Configurations and hyperparameters - Easy tuning with Ray Tune.
1. Fixing our experiment's config - Going multi-agent.
1. The "infinite laptop": Quick intro into how to use RLlib with Anyscale's product.
1. Exercise No.2 (run your own Ray RLlib+Tune experiment)
1. Neural network models - Provide your custom models using tf.keras or torch.nn.
1. Deeper dive into RLlib's parallelization architecture.
1. Specifying different compute resources and parallelization options through our config.
1. "Hacking in": Using callbacks to customize the RL loop and generate our own metrics.
1. Exercise No.3 (write your own custom callback)
1. "Hacking in (part II)" - Debugging with RLlib and PyCharm.
1. Checking on the "infinite laptop" - Did RLlib learn to solve the problem?

### Other Recommended Readings
* [Attention Nets and More with RLlib's Trajectory View API](https://medium.com/distributed-computing-with-ray/attention-nets-and-more-with-rllibs-trajectory-view-api-d326339a6e65)
* [Intro to RLlib: Example Environments](https://medium.com/distributed-computing-with-ray/intro-to-rllib-example-environments-3a113f532c70)
* [Reinforcement Learning with RLlib in the Unity Game Engine](https://medium.com/distributed-computing-with-ray/reinforcement-learning-with-rllib-in-the-unity-game-engine-1a98080a7c0d)


In [5]:
import numpy as np

import ray

# Start a new instance of Ray or connect to an already running one.
ray.init()
# In case you encounter this error during our tutorial:
# RuntimeError: Maybe you called ray.init twice by accident?
# Try: ray.shutdown() or ray.init(ignore_reinit_error=True)

2021-04-29 15:30:53,640	INFO services.py:1267 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '192.168.0.100',
 'raylet_ip_address': '192.168.0.100',
 'redis_address': '192.168.0.100:6379',
 'object_store_address': '/tmp/ray/session_2021-04-29_15-30-52_145625_27624/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-04-29_15-30-52_145625_27624/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2021-04-29_15-30-52_145625_27624',
 'metrics_export_port': 63009,
 'node_id': 'f9c56d30f3030d5011db17b9f583f5477ac374d682c8820c0477a48e'}

<img src="images/rl-cycle.png" width=1200>

In [6]:
# 2) Coding/defining our "problem" via an RL environment.
# We will use the following (adversarial) multi-agent environment
# throughout this tutorial to demonstrate a large fraction of RLlib's
# APIs, features, and customization options.

(raylet) /Users/sven/opt/anaconda3/envs/ray_tutorial/lib/python3.8/site-packages/ray/autoscaler/_private/cli_logger.py:57: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
(raylet)   warnings.warn(
(raylet) /Users/sven/opt/anaconda3/envs/ray_tutorial/lib/python3.8/site-packages/ray/autoscaler/_private/cli_logger.py:57: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
(raylet)   warnings.warn(
(raylet) /Users/sven/opt/anaconda3/envs/ray_tutorial/lib/python3.8/site-packages/ray/autoscaler/_private/cli_logger.py:57: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your in

<img src="images/environment.png" width=800>

### A word or two on Spaces:

Spaces are used in ML to describe what possible/valid values inputs and outputs of a neural network can have.

RL environments also use them to describe what their valid observations and actions are.

Spaces are usually defined by their shape (e.g. 84x84x3 RGB images) and datatype (e.g. uint8 for RGB values between 0 and 255).
However, spaces could also be composed of other spaces (see Tuple or Dict spaces) or could be simply discrete with n fixed possible values
(represented by integers). For example, in our game, where each agent can only go up/down/left/right, the action space would be "Discrete(4)"
(no datatype, no shape needs to be defined here).

<img src="images/spaces.png" width=800>

In [7]:
import gym
from gym.spaces import Discrete, MultiDiscrete
import random

from ray.rllib.env.multi_agent_env import MultiAgentEnv

class MultiAgentArena(MultiAgentEnv):
    def __init__(self, config=None):
        # !LIVE CODING!
        config = config or {}
        self.width = config.get("width", 10)
        self.height = config.get("height", 10)

        # 0=up, 1=right, 2=down, 3=left.
        self.action_space = Discrete(4)
        self.observation_space = MultiDiscrete([self.width * self.height,
                                                self.width * self.height])
        # End an episode after this many timesteps.
        self.timestep_limit = config.get("ts", 100)
        # Reset env.
        self.reset()

    def reset(self):
        # !LIVE CODING!
        # Row-major coords.
        self.agent1_pos = [0, 0]
        self.agent2_pos = [self.height - 1, self.width - 1]
        # Reset agent1's visited states.
        self.agent1_visited_states = set()
        # How many timesteps have we done in this episode.
        self.timesteps = 0

        return self.get_obs()

    def step(self, action: dict):
        # !LIVE CODING!
        self.timesteps += 1
        # Determine, who is allowed to move first.
        agent1_first = random.random() > 0.5
        # Move first agent (could be agent 1 or 2).
        if agent1_first:
            r1, r2 = self.move(self.agent1_pos, action["agent1"], is_agent1=True)
            add = self.move(self.agent2_pos, action["agent2"], is_agent1=False)
        else:
            r1, r2 = self.move(self.agent2_pos, action["agent2"], is_agent1=False)
            add = self.move(self.agent1_pos, action["agent1"], is_agent1=True)
        r1 += add[0]
        r2 += add[1]

        obs = self.get_obs()

        reward = {"agent1": r1, "agent2": r2}

        done = self.timesteps >= self.timestep_limit
        done = {"agent1": done, "agent2": done, "__all__": done}

        return obs, reward, done, {}

    def get_obs(self):
        ag1_discrete_pos = self.agent1_pos[0] * self.width + \
            (self.agent1_pos[1] % self.width)
        ag2_discrete_pos = self.agent2_pos[0] * self.width + \
            (self.agent2_pos[1] % self.width)
        return {
            "agent1": np.array([ag1_discrete_pos, ag2_discrete_pos]),
            "agent2": np.array([ag2_discrete_pos, ag1_discrete_pos]),
        }

    def move(self, coords, action, is_agent1):
        orig_coords = coords[:]
        # Change the row: 0=up (-1), 2=down (+1)
        coords[0] += -1 if action == 0 else 1 if action == 2 else 0
        # Change the column: 1=right (+1), 3=left (-1)
        coords[1] += 1 if action == 1 else -1 if action == 3 else 0

        # Solve collisions.
        # Make sure, we don't end up on the other agent's position.
        # If yes, don't move (we are blocked).
        if (is_agent1 and coords == self.agent2_pos) or (not is_agent1 and coords == self.agent1_pos):
            coords[0], coords[1] = orig_coords
            # Agent2 blocked agent1 (agent1 tried to run into agent2)
            # OR Agent2 bumped into agent1 (agent2 tried to run into agent1)
            # -> +1 for agent2; -1 for agent1
            return -1.0, 1.0

        # No agent blocking -> check walls.
        if coords[0] < 0:
            coords[0] = 0
        elif coords[0] >= self.height:
            coords[0] = self.height - 1
        if coords[1] < 0:
            coords[1] = 0
        elif coords[1] >= self.width:
            coords[1] = self.width - 1

        # If agent1 -> +1.0 if new tile covered.
        if is_agent1 and not tuple(coords) in self.agent1_visited_states:
            self.agent1_visited_states.add(tuple(coords))
            return 1.0, -0.1
        # No new tile for agent1 -> Negative reward.
        return -0.5, -0.1

    # Optionally: Add `render` method returning some img.
    def render(self, mode=None):
        return np.random.randint(0, 256, (20, 20, 3), dtype=np.uint8)

Instructions for updating:
non-resource variables are not supported in the long term


## Exercise No 1

Write an "environment loop" using our `MultiAgentArena` class.

1. Create an env object.
1. `reset` your environment to get the first (initial) observation.
1. `step` through the environment using a provided
   "DummyTrainer.compute_action([obs])" method to compute action dicts (see cell below, in which you can create a DummyTrainer object and query it for random actions).
1. When an episode is done, remember to `reset()` your environment before the next call to `step()`.
1. If you feel, this is way too easy for you ;) , try to extract each agent's reward, sum it up over one episode and - at the end of an episode (when done=True) - print out each agent's accumulated reward (also called "return").

**Good luck! :)**


In [8]:
class DummyTrainer:
    """Dummy Trainer class used in Exercise #1.

    Use its `compute_action` method to get a new action, given some environment
    observation.
    """

    def compute_action(self, obs):
        # Returns a random action.
        return {
            "agent1": np.random.randint(4),
            "agent2": np.random.randint(4)
        }

dummy_trainer = DummyTrainer()
# Check, whether it's working.
for _ in range(3):
    print(dummy_trainer.compute_action({"agent1": np.array([0, 10]), "agent2": np.array([10, 0])}))

{'agent1': 3, 'agent2': 1}
{'agent1': 1, 'agent2': 2}
{'agent1': 1, 'agent2': 0}


In [9]:
# Solution to Exercise #1:
#from gym.envs.classic_control.rendering import SimpleImageViewer
#simple_image_viewer = SimpleImageViewer()

# Solution:
env = MultiAgentArena(config={"width": 10, "height": 10})
obs = env.reset()
# Play through a single episode.
done = {"__all__": False}
return_ag1 = return_ag2 = 0.0
num_episodes = 0
while num_episodes < 10:
    action = dummy_trainer.compute_action(obs)
    obs, rewards, done, _ = env.step(action)
    return_ag1 += rewards["agent1"]
    return_ag2 += rewards["agent2"]    
    if done["__all__"]:
        print(f"Episode done. R1={return_ag1} R2={return_ag2}")
        num_episodes += 1
        return_ag1 = return_ag2 = 0.0
        obs = env.reset()
    # Optional:
    #img = env.render()
    #simple_image_viewer.imshow(img)


Episode done. R1=-60.0 R2=-18.899999999999963
Episode done. R1=-48.5 R2=-17.79999999999996
Episode done. R1=-50.5 R2=-16.699999999999964
Episode done. R1=-68.5 R2=-19.99999999999996
Episode done. R1=-80.0 R2=-11.19999999999997
Episode done. R1=-45.5 R2=-14.499999999999972
Episode done. R1=-53.5 R2=-19.99999999999996
Episode done. R1=-59.5 R2=-19.99999999999996
Episode done. R1=-51.0 R2=-15.599999999999962
Episode done. R1=-46.0 R2=-19.99999999999996


In [10]:
# 4) Plugging in RLlib.

# Import a Trainable (one of RLlib's built-in algorithms):
# We use the PPO algorithm here b/c its very flexible wrt its supported
# action spaces and model types and b/c it learns well almost any problem.
from ray.rllib.agents.ppo import PPOTrainer

# Specify a very simple config, defining our environment and some environment
# options (see environment.py).
config = {
    "env": MultiAgentArena,
    "env_config": {
        "config": {
            "width": 10,
            "height": 10,
        },
    },
    # "framework": "torch",
    "create_env_on_driver": True,
}
# Instantiate the Trainer object using above config.
rllib_trainer = PPOTrainer(config=config)


2021-04-29 15:31:12,852	INFO trainer.py:669 -- Tip: set framework=tfe or the --eager flag to enable TensorFlow eager execution
2021-04-29 15:31:12,853	INFO trainer.py:694 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=27764) WARNING:tensorflow:From /Users/sven/opt/anaconda3/envs/ray_tutorial/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=27764) Instructions for updating:
(pid=27764) non-resource variables are not supported in the long term
(pid=27764) WARNING:tensorflow:From /Users/sven/opt/anaconda3/envs/ray_tutorial/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=27764) Instructions for updating:
(pid=27764) no

In [11]:
# That's it, we are ready to train.
# Calling `train` once runs a single "training iteration". One iteration
# for most algos contains a) sampling from the environment(s) + b) using the
# sampled data (observations, actions taken, rewards) to update the policy
# model (neural network), such that it would pick better actions in the future,
# leading to higher rewards.
print(rllib_trainer.train())

2021-04-29 15:31:19,175	WARNING deprecation.py:33 -- DeprecationWarning: `SampleBatch.data[..]` has been deprecated. Use `SampleBatch[..]` instead. This will raise an error in the future!


{'episode_reward_max': -35.999999999999986, 'episode_reward_min': -88.50000000000016, 'episode_reward_mean': -65.5800000000001, 'episode_len_mean': 100.0, 'episode_media': {}, 'episodes_this_iter': 20, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [-52.50000000000007, -60.00000000000004, -76.80000000000015, -78.60000000000014, -70.50000000000016, -52.50000000000004, -84.00000000000011, -58.50000000000006, -76.50000000000014, -35.999999999999986, -63.00000000000006, -72.00000000000011, -64.50000000000014, -70.50000000000011, -70.20000000000007, -78.00000000000011, -47.70000000000003, -88.50000000000016, -55.8000000000001, -55.50000000000007], 'episode_lengths': [100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100]}, 'sampler_perf': {'mean_raw_obs_processing_ms': 0.1742773122720785, 'mean_inference_ms': 0.6968317689238253, 'mean_action_processing_ms': 0.0682

In [16]:
# Run `train()` n times. Try to repeatedly call this to see rewards increase.
# Move on once you see episode rewards > -55.0.
for i in range(10):
    results = rllib_trainer.train()
    print(f"iteration {i}: R={results['episode_reward_mean']}")

iteration 0: R=-56.406000000000056
iteration 1: R=-56.043000000000056
iteration 2: R=-54.92700000000004
iteration 3: R=-53.10300000000006
iteration 4: R=-52.950000000000045
iteration 5: R=-51.99300000000006
iteration 6: R=-51.91500000000005
iteration 7: R=-52.30500000000006
iteration 8: R=-53.940000000000055
iteration 9: R=-53.89200000000005


In [17]:
# !LIVE CODING!
# Let's actually "look inside" our Trainer to see what's in there.
pol = rllib_trainer.get_policy()
print(f"Policy: {pol}; Observation-space: {pol.observation_space}; Action-space: {pol.action_space}")

print(f"Model: {pol.model}")

# Create a fake numpy B=1 (single) observation consisting of both agents positions ("one-hot'd" and "concat'd").
from ray.rllib.utils.numpy import one_hot
single_obs = np.concatenate([one_hot(0, depth=100), one_hot(99, depth=100)])
single_obs = np.array([single_obs])
#single_obs.shape

# Generate the Model's output.
out, state_out = pol.model({"obs": single_obs})

# tf1.x (static graph) -> Need to run this through a tf session.
numpy_out = pol._sess.run(out)

# RLlib then passes the model's output to the policy's "action distribution" to sample an action.
action_dist = pol.dist_class(out)
action = action_dist.sample()

# Show us the actual action.
pol._sess.run(action)

Policy: <ray.rllib.policy.tf_policy_template.PPOTFPolicy object at 0x7ffc05b10670>; Observation-space: Box(-1.0, 1.0, (200,), float32); Action-space: Discrete(4)
Model: <ray.rllib.models.tf.fcnet.FullyConnectedNetwork object at 0x7ffc05b106d0>


array([1])

In [18]:
# Save our trainer.
checkpoint_path = rllib_trainer.save()
print(f"Trainer was saved in '{checkpoint_path}'!")

import os
os.listdir(os.path.dirname(checkpoint_path))

Trainer was saved in '/Users/sven/ray_results/PPO_MultiAgentArena_2021-04-29_15-31-12gg_j3w3g/checkpoint_000041/checkpoint-41'!


['checkpoint-41.tune_metadata', 'checkpoint-41', '.is_checkpoint']

In [19]:
# Pretend, we wanted to pick up training from a previous run:
new_trainer = PPOTrainer(config=config)
# Evaluate the new trainer (this should yield random results).
results = new_trainer._evaluate()
print(f"Evaluating new trainer: R={results['evaluation']['episode_reward_mean']}")

# Restoring the trained state into the `new_trainer` object.
new_trainer.restore(checkpoint_path)

# Evaluate again (this should yield results we saw after having trained our saved agent).
results = new_trainer._evaluate()
print(f"Evaluating restored trainer: R={results['evaluation']['episode_reward_mean']}")

(pid=27766) WARNING:tensorflow:From /Users/sven/opt/anaconda3/envs/ray_tutorial/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=27766) Instructions for updating:
(pid=27766) non-resource variables are not supported in the long term
(pid=27766) WARNING:tensorflow:From /Users/sven/opt/anaconda3/envs/ray_tutorial/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=27766) Instructions for updating:
(pid=27766) non-resource variables are not supported in the long term
(pid=27760) WARNING:tensorflow:From /Users/sven/opt/anaconda3/envs/ray_tutorial/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprec

AttributeError: 'PPO' object has no attribute 'evaluation_workers'

In [20]:
# 5) Configuration dicts and Ray Tune.
# Where are the default configuration dicts stored?
import pprint
from ray.rllib.agents.ppo import DEFAULT_CONFIG as PPO_DEFAULT_CONFIG
print(f"PPO's default config is:")
pprint.pprint(PPO_DEFAULT_CONFIG)

#from ray.rllib.agents.dqn import DEFAULT_CONFIG as DQN_DEFAULT_CONFIG
#print(f"DQN's default config is:")
#pprint.pprint(DQN_DEFAULT_CONFIG)

#from ray.rllib.agents.trainer import COMMON_CONFIG
#print(f"RLlib Trainer's default config is:")
#pprint.pprint(COMMON_CONFIG)

PPO's default config is:
{'_fake_gpus': False,
 'batch_mode': 'truncate_episodes',
 'callbacks': <class 'ray.rllib.agents.callbacks.DefaultCallbacks'>,
 'clip_actions': True,
 'clip_param': 0.3,
 'clip_rewards': None,
 'collect_metrics_timeout': 180,
 'compress_observations': False,
 'create_env_on_driver': False,
 'custom_eval_function': None,
 'custom_resources_per_worker': {},
 'eager_tracing': False,
 'entropy_coeff': 0.0,
 'entropy_coeff_schedule': None,
 'env': None,
 'env_config': {},
 'evaluation_config': {},
 'evaluation_interval': None,
 'evaluation_num_episodes': 10,
 'evaluation_num_workers': 0,
 'exploration_config': {'type': 'StochasticSampling'},
 'explore': True,
 'extra_python_environs_for_driver': {},
 'extra_python_environs_for_worker': {},
 'fake_sampler': False,
 'framework': 'tf',
 'gamma': 0.99,
 'grad_clip': None,
 'horizon': None,
 'ignore_worker_failures': False,
 'in_evaluation': False,
 'input': 'sampler',
 'input_evaluation': ['is', 'wis'],
 'kl_coeff': 0.2

In [21]:
# Plugging in Ray Tune.
# Note that this is the recommended way to run any experiments with RLlib.
# Reasons:
# - Tune allows you to do hyperparameter tuning in a user-friendly way
#   and at large scale!
# - Tune automatically allocates needed resources for the different
#   hyperparam trials and experiment runs.

from ray import tune

# Now that we will run things "automatically" through tune, we have to
# define one or more stopping criteria.
stop = {
    # explain that keys here can be anything present in the above print(trainer.train())
    "training_iteration": 5,
    "episode_reward_mean": 9999.9,
}

# "PPO" is a registered name that points to RLlib's PPOTrainer.
# See `ray/rllib/agents/registry.py`
# Run our simple experiment until one of the stop criteria is met.
tune.run("PPO", config=config, stop=stop)


Trial name,status,loc
PPO_MultiAgentArena_4525b_00000,PENDING,


(pid=27758) WARNING:tensorflow:From /Users/sven/opt/anaconda3/envs/ray_tutorial/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=27758) Instructions for updating:
(pid=27758) non-resource variables are not supported in the long term
(pid=27758) WARNING:tensorflow:From /Users/sven/opt/anaconda3/envs/ray_tutorial/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=27758) Instructions for updating:
(pid=27758) non-resource variables are not supported in the long term
(pid=27758) 2021-04-29 15:39:12,833	INFO trainer.py:669 -- Tip: set framework=tfe or the --eager flag to enable TensorFlow eager execution
(pid=27758) 2021-04-29 15:39:12,833	INFO trainer.py:694 -- Current log_level is WARN. For m

Result for PPO_MultiAgentArena_4525b_00000:
  agent_timesteps_total: 4000
  custom_metrics: {}
  date: 2021-04-29_15-39-22
  done: false
  episode_len_mean: 100.0
  episode_media: {}
  episode_reward_max: -43.800000000000026
  episode_reward_mean: -70.08000000000011
  episode_reward_min: -86.40000000000015
  episodes_this_iter: 20
  episodes_total: 20
  experiment_id: 8553c35b91814519ad2ca12c7fa11668
  hostname: Svens-MacBook-Pro.local
  info:
    learner:
      default_policy:
        learner_stats:
          cur_kl_coeff: 0.20000000298023224
          cur_lr: 4.999999873689376e-05
          entropy: 1.362125039100647
          entropy_coeff: 0.0
          kl: 0.02420918457210064
          model: {}
          policy_loss: -0.059532247483730316
          total_loss: 133.79750061035156
          vf_explained_var: 0.0809241235256195
          vf_loss: 133.8521728515625
    num_agent_steps_sampled: 4000
    num_steps_sampled: 4000
    num_steps_trained: 4000
  iterations_since_restore: 1


Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_MultiAgentArena_4525b_00000,RUNNING,192.168.0.100:27758,1,3.91841,4000,-70.08,-43.8,-86.4,100


Result for PPO_MultiAgentArena_4525b_00000:
  agent_timesteps_total: 12000
  custom_metrics: {}
  date: 2021-04-29_15-39-27
  done: false
  episode_len_mean: 100.0
  episode_media: {}
  episode_reward_max: -35.99999999999999
  episode_reward_mean: -62.04000000000009
  episode_reward_min: -88.50000000000014
  episodes_this_iter: 20
  episodes_total: 60
  experiment_id: 8553c35b91814519ad2ca12c7fa11668
  hostname: Svens-MacBook-Pro.local
  info:
    learner:
      default_policy:
        learner_stats:
          cur_kl_coeff: 0.30000001192092896
          cur_lr: 4.999999873689376e-05
          entropy: 1.3045539855957031
          entropy_coeff: 0.0
          kl: 0.01824115589261055
          model: {}
          policy_loss: -0.05359369516372681
          total_loss: 66.01841735839844
          vf_explained_var: 0.2051105946302414
          vf_loss: 66.06654357910156
    num_agent_steps_sampled: 12000
    num_steps_sampled: 12000
    num_steps_trained: 12000
  iterations_since_restore: 

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_MultiAgentArena_4525b_00000,RUNNING,192.168.0.100:27758,3,9.27795,12000,-62.04,-36,-88.5,100


Result for PPO_MultiAgentArena_4525b_00000:
  agent_timesteps_total: 20000
  custom_metrics: {}
  date: 2021-04-29_15-39-32
  done: true
  episode_len_mean: 100.0
  episode_media: {}
  episode_reward_max: -35.99999999999999
  episode_reward_mean: -61.71000000000009
  episode_reward_min: -88.50000000000016
  episodes_this_iter: 20
  episodes_total: 100
  experiment_id: 8553c35b91814519ad2ca12c7fa11668
  hostname: Svens-MacBook-Pro.local
  info:
    learner:
      default_policy:
        learner_stats:
          cur_kl_coeff: 0.30000001192092896
          cur_lr: 4.999999873689376e-05
          entropy: 1.2382826805114746
          entropy_coeff: 0.0
          kl: 0.01850685104727745
          model: {}
          policy_loss: -0.04642891883850098
          total_loss: 71.22037506103516
          vf_explained_var: 0.23586240410804749
          vf_loss: 71.2612533569336
    num_agent_steps_sampled: 20000
    num_steps_sampled: 20000
    num_steps_trained: 20000
  iterations_since_restore: 

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_MultiAgentArena_4525b_00000,TERMINATED,,5,14.2496,20000,-61.71,-36,-88.5,100


Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_MultiAgentArena_4525b_00000,TERMINATED,,5,14.2496,20000,-61.71,-36,-88.5,100


2021-04-29 15:39:33,025	INFO tune.py:549 -- Total run time: 25.97 seconds (25.42 seconds for the tuning loop).


In [22]:
# Updating an algo's default config dict and adding hyperparameter tuning
# options to it.
# Note: Hyperparameter tuning options (e.g. grid_search) will only work,
# if we run these configs via `tune.run`.
config.update(
    {
        # Try 2 different learning rates.
        "lr": tune.grid_search([0.0001, 0.5]),
        # NN model config to tweak the default model
        # that'll be created by RLlib for the policy.
        "model": {
            # e.g. change the dense layer stack.
            "fcnet_hiddens": [256, 256, 256],
            # Alternatively, you can specify a custom model here
            # (we'll cover that later).
            # "custom_model": ...
            # Pass kwargs to your custom model.
            # "custom_model_config": {}
        },
    }
)
# Repeat our experiment using tune's grid-search feature.
results = tune.run(
    "PPO",
    config=config,
    stop=stop,
    checkpoint_at_end=True,  # create a checkpoint when done.
    checkpoint_freq=1,  # create a checkpoint on every iteration.
)
print(results)


Trial name,status,loc,lr
PPO_MultiAgentArena_54a34_00000,PENDING,,0.0001
PPO_MultiAgentArena_54a34_00001,PENDING,,0.5


(pid=27753) WARNING:tensorflow:From /Users/sven/opt/anaconda3/envs/ray_tutorial/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=27753) Instructions for updating:
(pid=27753) non-resource variables are not supported in the long term
(pid=27753) WARNING:tensorflow:From /Users/sven/opt/anaconda3/envs/ray_tutorial/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=27753) Instructions for updating:
(pid=27753) non-resource variables are not supported in the long term
(pid=27755) WARNING:tensorflow:From /Users/sven/opt/anaconda3/envs/ray_tutorial/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprec

Result for PPO_MultiAgentArena_54a34_00001:
  agent_timesteps_total: 4000
  custom_metrics: {}
  date: 2021-04-29_15-39-47
  done: false
  episode_len_mean: 100.0
  episode_media: {}
  episode_reward_max: -48.30000000000009
  episode_reward_mean: -66.69000000000011
  episode_reward_min: -93.00000000000013
  episodes_this_iter: 20
  episodes_total: 20
  experiment_id: 3898debe393b45bba3ba5505420443c5
  hostname: Svens-MacBook-Pro.local
  info:
    learner:
      default_policy:
        learner_stats:
          cur_kl_coeff: 0.20000000298023224
          cur_lr: 0.5
          entropy: 0.03141152858734131
          entropy_coeff: 0.0
          kl: 36.357173919677734
          model: {}
          policy_loss: 0.4721342921257019
          total_loss: 158.54925537109375
          vf_explained_var: -0.0006072241812944412
          vf_loss: 150.80569458007812
    num_agent_steps_sampled: 4000
    num_steps_sampled: 4000
    num_steps_trained: 4000
  iterations_since_restore: 1
  node_ip: 192.1

Trial name,status,loc,lr,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_MultiAgentArena_54a34_00000,RUNNING,,0.0001,,,,,,,
PPO_MultiAgentArena_54a34_00001,RUNNING,192.168.0.100:27755,0.5,1,4.33747,4000,-66.69,-48.3,-93,100


Result for PPO_MultiAgentArena_54a34_00000:
  agent_timesteps_total: 4000
  custom_metrics: {}
  date: 2021-04-29_15-39-47
  done: false
  episode_len_mean: 100.0
  episode_media: {}
  episode_reward_max: -56.4000000000001
  episode_reward_mean: -71.61000000000011
  episode_reward_min: -88.50000000000017
  episodes_this_iter: 20
  episodes_total: 20
  experiment_id: e48d71458a3945f4bcc1000b5dec669e
  hostname: Svens-MacBook-Pro.local
  info:
    learner:
      default_policy:
        learner_stats:
          cur_kl_coeff: 0.20000000298023224
          cur_lr: 9.999999747378752e-05
          entropy: 1.345606803894043
          entropy_coeff: 0.0
          kl: 0.041982509195804596
          model: {}
          policy_loss: -0.07474327087402344
          total_loss: 138.66407775878906
          vf_explained_var: 0.1261296421289444
          vf_loss: 138.73043823242188
    num_agent_steps_sampled: 4000
    num_steps_sampled: 4000
    num_steps_trained: 4000
  iterations_since_restore: 1
 

...

Result for PPO_MultiAgentArena_54a34_00000:
  agent_timesteps_total: 12000
  custom_metrics: {}
  date: 2021-04-29_15-39-56
  done: false
  episode_len_mean: 100.0
  episode_media: {}
  episode_reward_max: -47.70000000000005
  episode_reward_mean: -68.0600000000001
  episode_reward_min: -90.90000000000015
  episodes_this_iter: 20
  episodes_total: 60
  experiment_id: e48d71458a3945f4bcc1000b5dec669e
  hostname: Svens-MacBook-Pro.local
  info:
    learner:
      default_policy:
        learner_stats:
          cur_kl_coeff: 0.44999998807907104
          cur_lr: 9.999999747378752e-05
          entropy: 1.2796211242675781
          entropy_coeff: 0.0
          kl: 0.02720813825726509
          model: {}
          policy_loss: -0.06847807765007019
          total_loss: 64.20436096191406
          vf_explained_var: 0.3745890259742737
          vf_loss: 64.26058959960938
    num_agent_steps_sampled: 12000
    num_steps_sampled: 12000
    num_steps_trained: 12000
  iterations_since_restore: 3

Trial name,status,loc,lr,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_MultiAgentArena_54a34_00000,RUNNING,192.168.0.100:27753,0.0001,5,20.5724,20000,-66.06,-44.7,-90.9,100
PPO_MultiAgentArena_54a34_00001,RUNNING,192.168.0.100:27755,0.5,4,16.5365,16000,-96.5475,-48.3,-106.5,100


Result for PPO_MultiAgentArena_54a34_00001:
  agent_timesteps_total: 20000
  custom_metrics: {}
  date: 2021-04-29_15-40-04
  done: true
  episode_len_mean: 100.0
  episode_media: {}
  episode_reward_max: -48.30000000000009
  episode_reward_mean: -98.53800000000018
  episode_reward_min: -106.50000000000017
  episodes_this_iter: 20
  episodes_total: 100
  experiment_id: 3898debe393b45bba3ba5505420443c5
  hostname: Svens-MacBook-Pro.local
  info:
    learner:
      default_policy:
        learner_stats:
          cur_kl_coeff: 0.03750000149011612
          cur_lr: 0.5
          entropy: 0.0
          entropy_coeff: 0.0
          kl: 0.0
          model: {}
          policy_loss: -0.004698052071034908
          total_loss: 424.4887390136719
          vf_explained_var: -1.6763806343078613e-08
          vf_loss: 424.4934387207031
    num_agent_steps_sampled: 20000
    num_steps_sampled: 20000
    num_steps_trained: 20000
  iterations_since_restore: 5
  node_ip: 192.168.0.100
  num_healthy_w

Trial name,status,loc,lr,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_MultiAgentArena_54a34_00000,TERMINATED,,0.0001,5,20.5724,20000,-66.06,-44.7,-90.9,100
PPO_MultiAgentArena_54a34_00001,TERMINATED,,0.5,5,20.6172,20000,-98.538,-48.3,-106.5,100


(pid=27754) 2021-04-29 15:40:04,397	ERROR worker.py:382 -- SystemExit was raised from the worker
(pid=27754) Traceback (most recent call last):
(pid=27754)   File "python/ray/_raylet.pyx", line 488, in ray._raylet.execute_task
(pid=27754)   File "python/ray/_raylet.pyx", line 495, in ray._raylet.execute_task
(pid=27754)   File "python/ray/_raylet.pyx", line 505, in ray._raylet.execute_task
(pid=27754)   File "python/ray/_raylet.pyx", line 449, in ray._raylet.execute_task.function_executor
(pid=27754)   File "/Users/sven/opt/anaconda3/envs/ray_tutorial/lib/python3.8/site-packages/ray/_private/function_manager.py", line 556, in actor_method_executor
(pid=27754)     return method(__ray_actor, *args, **kwargs)
(pid=27754)   File "/Users/sven/opt/anaconda3/envs/ray_tutorial/lib/python3.8/site-packages/ray/actor.py", line 1001, in __ray_terminate__
(pid=27754)     ray.actor.exit_actor()
(pid=27754)   File "/Users/sven/opt/anaconda3/envs/ray_tutorial/lib/python3.8/site-packages/ray/actor.py",

In [23]:
# 6) Going multi-policy: Our experiment is ill-configured b/c both
# agents, which should behave differently due to their different
# tasks and reward functions, learn the same policy (the "default_policy",
# which RLlib always provides if you don't configure anything else; Remember
# that RLlib does not know at Trainer setup time, how many and which agents
# the environment will "produce").
# Let's fix this and introduce the "multiagent" API.

# 6.1.) Define an agent->policy mapping function.
# Which agents (defined by the environment) use which policies
# (defined by us)? Mapping is M (agents) -> N (policies), where M >= N.
def policy_mapping_fn(agent: str):
    assert agent in ["agent1", "agent2"], f"ERROR: invalid agent {agent}!"
    return "pol1" if agent == "agent1" else "pol2"
    
# 6.2.) Define details for our two policies.
#TODO: coding Sven: Make it possible to not need obs/action spaces
#  if they are the default anyways.
observation_space = rllib_trainer.workers.local_worker().env.observation_space
action_space = rllib_trainer.workers.local_worker().env.action_space
# Btw, the above is equivalent to saying:
# >>> rllib_trainer.get_policy("default_policy").obs/action_space
policies = {
    "pol1": (None, observation_space, action_space, {"lr": 0.0003}),
    "pol2": (None, observation_space, action_space, {"lr": 0.0004}),
}

#policies_to_train = ["pol1", "pol2"]

# 6.3) Adding the above to our config.
config.update({
    "multiagent": {
        "policies": policies,
        "policy_mapping_fn": policy_mapping_fn,
        #"policies_to_train": policies_to_train,
    },
})


## Exercise No 2

Try learning our environment using Ray tune.run and a simple hyperparameter grid_search over:
- 2 different learning rates (pick your own values).
- AND 2 different `train_batch_size` settings (use 2000 and 3000).

Also, make RLlib use a [128, 128] dense layer stack as the NN model.

Also, use the config setting of `num_envs_per_worker=10` to increase the sampling throughput.

In case your local machine has less than 12 CPUs, try setting `num_workers=1` to make all tune trials run at the same time.
Background: PPO by default uses 2 workers, which makes 1 trial use 3 CPUs (2 workers + "driver" ("local-worker")),
which makes the entire experiment use 12 CPUs. Tune will run trials in sequence in case it cannot allocate enough CPUs at once
(which is also fine, but then takes longer).

Try to reach a total reward (sum of agent1 and agent2) of -25.0.

**Good luck! :)**


In [24]:
# Solution to Exercise #2:

# Update our config and set it up for 2x tune grid-searches (leading to 4 parallel trials in total).
config.update({
    "lr": tune.grid_search([0.0001, 0.0005]),
    "train_batch_size": tune.grid_search([2000, 3000]),
    "num_envs_per_worker": 10,
    # Change our model to be simpler.
    "model": {
        "fcnet_hiddens": [128, 128],
    },
})

# Run the experiment.
tune.run("PPO", config=config, stop={"episode_reward_mean": -25.0, "training_iteration": 100})

Trial name,status,loc,lr,train_batch_size
PPO_MultiAgentArena_6768d_00000,PENDING,,0.0001,2000
PPO_MultiAgentArena_6768d_00001,PENDING,,0.0005,2000
PPO_MultiAgentArena_6768d_00002,PENDING,,0.0001,3000
PPO_MultiAgentArena_6768d_00003,PENDING,,0.0005,3000


(raylet) /Users/sven/opt/anaconda3/envs/ray_tutorial/lib/python3.8/site-packages/ray/autoscaler/_private/cli_logger.py:57: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
(raylet)   warnings.warn(
(raylet) /Users/sven/opt/anaconda3/envs/ray_tutorial/lib/python3.8/site-packages/ray/autoscaler/_private/cli_logger.py:57: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
(raylet)   warnings.warn(
(raylet) /Users/sven/opt/anaconda3/envs/ray_tutorial/lib/python3.8/site-packages/ray/autoscaler/_private/cli_logger.py:57: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your in

(raylet) /Users/sven/opt/anaconda3/envs/ray_tutorial/lib/python3.8/site-packages/ray/autoscaler/_private/cli_logger.py:57: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
(raylet)   warnings.warn(
(raylet) /Users/sven/opt/anaconda3/envs/ray_tutorial/lib/python3.8/site-packages/ray/autoscaler/_private/cli_logger.py:57: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
(raylet)   warnings.warn(
(raylet) /Users/sven/opt/anaconda3/envs/ray_tutorial/lib/python3.8/site-packages/ray/autoscaler/_private/cli_logger.py:57: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your in

(pid=27751) 2021-04-29 15:40:18,899	WARNING util.py:53 -- Install gputil for GPU system monitoring.
(pid=27751) 2021-04-29 15:40:18,899	WARNING util.py:53 -- Install gputil for GPU system monitoring.
(pid=27761) 2021-04-29 15:40:19,289	WARNING util.py:53 -- Install gputil for GPU system monitoring.
(pid=27761) 2021-04-29 15:40:19,289	WARNING util.py:53 -- Install gputil for GPU system monitoring.
(pid=27831) 2021-04-29 15:40:19,282	WARNING util.py:53 -- Install gputil for GPU system monitoring.
(pid=27831) 2021-04-29 15:40:19,282	WARNING util.py:53 -- Install gputil for GPU system monitoring.
(pid=27752) 2021-04-29 15:40:19,307	WARNING util.py:53 -- Install gputil for GPU system monitoring.
(pid=27752) 2021-04-29 15:40:19,307	WARNING util.py:53 -- Install gputil for GPU system monitoring.
(pid=27751) 2021-04-29 15:40:19,579	WARNING deprecation.py:33 -- DeprecationWarning: `SampleBatch.data[..]` has been deprecated. Use `SampleBatch[..]` instead. This will raise an error in the future!


Result for PPO_MultiAgentArena_6768d_00000:
  agent_timesteps_total: 8000
  custom_metrics: {}
  date: 2021-04-29_15-40-24
  done: false
  episode_len_mean: 100.0
  episode_media: {}
  episode_reward_max: -45.90000000000006
  episode_reward_mean: -69.2925000000001
  episode_reward_min: -91.20000000000014
  episodes_this_iter: 40
  episodes_total: 40
  experiment_id: ca7d5fe006ec4c509dd419cd353bfbf5
  hostname: Svens-MacBook-Pro.local
  info:
    learner:
      pol1:
        learner_stats:
          cur_kl_coeff: 0.20000000298023224
          cur_lr: 0.0003000000142492354
          entropy: 1.3394379615783691
          entropy_coeff: 0.0
          kl: 0.04784049838781357
          model: {}
          policy_loss: -0.07369829714298248
          total_loss: 226.8167724609375
          vf_explained_var: 0.05389287695288658
          vf_loss: 226.88088989257812
      pol2:
        learner_stats:
          cur_kl_coeff: 0.20000000298023224
          cur_lr: 0.00039999998989515007
          e

...

Trial name,status,loc,lr,train_batch_size,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_MultiAgentArena_6768d_00003,RUNNING,192.168.0.100:27831,0.0005,3000,87,460.319,348000,-26.067,-0.3,-59.4,100
PPO_MultiAgentArena_6768d_00000,TERMINATED,,0.0001,2000,74,418.431,296000,-24.723,-8.4,-47.4,100
PPO_MultiAgentArena_6768d_00001,TERMINATED,,0.0005,2000,57,341.737,228000,-24.663,-6.9,-47.1,100
PPO_MultiAgentArena_6768d_00002,TERMINATED,,0.0001,3000,65,381.123,260000,-23.769,-4.5,-42,100


Result for PPO_MultiAgentArena_6768d_00003:
  agent_timesteps_total: 704000
  custom_metrics: {}
  date: 2021-04-29_15-48-06
  done: false
  episode_len_mean: 100.0
  episode_media: {}
  episode_reward_max: -0.29999999999999544
  episode_reward_mean: -25.682999999999993
  episode_reward_min: -59.10000000000005
  episodes_this_iter: 40
  episodes_total: 3520
  experiment_id: 57a3943dc3314ef292b59a6778e27a63
  hostname: Svens-MacBook-Pro.local
  info:
    learner:
      pol1:
        learner_stats:
          cur_kl_coeff: 1.5187499523162842
          cur_lr: 0.0003000000142492354
          entropy: 0.4750180244445801
          entropy_coeff: 0.0
          kl: 0.007025967352092266
          model: {}
          policy_loss: -0.026259679347276688
          total_loss: 41.6136589050293
          vf_explained_var: 0.23172453045845032
          vf_loss: 41.629249572753906
      pol2:
        learner_stats:
          cur_kl_coeff: 1.0125000476837158
          cur_lr: 0.00039999998989515007
    

Trial name,status,loc,lr,train_batch_size,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_MultiAgentArena_6768d_00000,TERMINATED,,0.0001,2000,74,418.431,296000,-24.723,-8.4,-47.4,100
PPO_MultiAgentArena_6768d_00001,TERMINATED,,0.0005,2000,57,341.737,228000,-24.663,-6.9,-47.1,100
PPO_MultiAgentArena_6768d_00002,TERMINATED,,0.0001,3000,65,381.123,260000,-23.769,-4.5,-42,100
PPO_MultiAgentArena_6768d_00003,TERMINATED,,0.0005,3000,89,466.944,356000,-24.78,-0.3,-55.2,100


Trial name,status,loc,lr,train_batch_size,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_MultiAgentArena_6768d_00000,TERMINATED,,0.0001,2000,74,418.431,296000,-24.723,-8.4,-47.4,100
PPO_MultiAgentArena_6768d_00001,TERMINATED,,0.0005,2000,57,341.737,228000,-24.663,-6.9,-47.1,100
PPO_MultiAgentArena_6768d_00002,TERMINATED,,0.0001,3000,65,381.123,260000,-23.769,-4.5,-42,100
PPO_MultiAgentArena_6768d_00003,TERMINATED,,0.0005,3000,89,466.944,356000,-24.78,-0.3,-55.2,100


(pid=27831) Exception ignored in: <function Connection.__del__ at 0x7fe49c83e940>
(pid=27831) Traceback (most recent call last):
(pid=27831)   File "/Users/sven/opt/anaconda3/envs/ray_tutorial/lib/python3.8/site-packages/redis/connection.py", line 543, in __del__
(pid=27831)     try:
(pid=27831)   File "/Users/sven/opt/anaconda3/envs/ray_tutorial/lib/python3.8/site-packages/ray/worker.py", line 379, in sigterm_handler
(pid=27831)     sys.exit(1)
(pid=27831) SystemExit: 1
(pid=27831) Traceback (most recent call last):
(pid=27831)   File "python/ray/_raylet.pyx", line 488, in ray._raylet.execute_task
(pid=27831)   File "python/ray/_raylet.pyx", line 495, in ray._raylet.execute_task
(pid=27831)   File "python/ray/_raylet.pyx", line 505, in ray._raylet.execute_task
(pid=27831)   File "python/ray/_raylet.pyx", line 449, in ray._raylet.execute_task.function_executor
(pid=27831)   File "/Users/sven/opt/anaconda3/envs/ray_tutorial/lib/python3.8/site-packages/ray/_private/function_manager.py", 

In [ ]:
# 8) Infinite laptop:

# NOTE: The following cell will only work if you are already on-boarded to our Anyscale Inc. "Infinite Laptop".
# To get more information, see https://www.anyscale.com/product

# Let's quickly divert from our MultiAgentArena and move to something much heavier in terms of environment/simulator complexity.
# We will now demonstrate, how you can use Anyscale's infinite laptop to launch an RLlib experiment on a cloud 4 GPU + 32 CPU machine
# all from within this Jupyter cell here.
# Start an experiment in the cloud using Anyscale's product, RLlib, and a more complex multi-agent env.

# NOTE 
import anyscale



In [ ]:
# 9) Custom Neural Network Models.

import tensorflow as tf


class MyModel(tf.keras.Model):
    def __init__(self,
                input_space,
                action_space,
                num_outputs,
                name="",
                *,
                layers = (256, 256)):
        super().__init__(name=name)

        self.dense_layers = []
        for i, layer_size in enumerate(layers):
            self.dense_layers.append(tf.keras.layers.Dense(
                layer_size, activation=tf.nn.relu, name=f"dense_{i}"))

        self.logits = tf.keras.layers.Dense(
            num_outputs,
            activation=tf.keras.activations.linear,
            name="logits")
        self.values = tf.keras.layers.Dense(
            1, activation=None, name="values")

    def call(self, inputs, training=None, mask=None):
        # Standardized input args:
        # - input_dict (RLlib `SampleBatch` object, which is basically a dict with numpy arrays
        # in it)
        out = inputs["obs"]
        for l in self.dense_layers:
            out = l(out)
        logits = self.logits(out)
        values = self.values(out)

        # Standardized output:
        # - "normal" model output tensor (e.g. action logits).
        # - list of internal state outputs (only needed for RNN-/memory enhanced models).
        # - "extra outs", such as model's side branches, e.g. value function outputs.
        return logits, [], {"vf_preds": tf.reshape(values, [-1])}

# Do a quick test.
from gym.spaces import Box
test_model = MyModel(
    input_space=Box(-1.0, 1.0, (2, )),
    action_space=None,
    num_outputs=2,
)
test_model({"obs": np.array([[0.5, 0.5]])})

In [ ]:
# "Hacking in": How do we customize our RL loop?
# RLlib offers a callbacks API that allows you to add custom behavior at
# all major events during the environment sampling and learning process.

# Our problem: So far, we can only see the total reward (sum for both agents).
# This does not give us enough insights into the question of which agent
# learns what (maybe agent2 doesn't learn anything and the reward we are observing
# is mostly due to agent1's progress in covering the map!).
# The following custom callbacks class allows us to add each agents single reward to
# the returned metrics, which will then be displayed in tensorboard.

# We will override RLlib's DefaultCallbacks class and implement the
# `on_episode_step` and `on_episode_end` methods therein.

from ray.rllib.agents.callbacks import DefaultCallbacks


class MyCallbacks(DefaultCallbacks):
    def on_episode_start(self, *, worker, base_env,
                         policies, episode,
                         env_index, **kwargs):
        episode.user_data["agent1_rewards"] = []
        episode.user_data["agent2_rewards"] = []

    def on_episode_step(self, *, worker, base_env,
                        episode, env_index, **kwargs):
        # Make sure this episode is ongoing.
        assert episode.length > 0, \
            "ERROR: `on_episode_step()` callback should not be called right " \
            "after env reset!"
        ag1_r = episode.prev_reward_for("agent1")
        ag2_r = episode.prev_reward_for("agent2")
        #print("ag1_r={} ag2_r={}".format(ag1_r, ag2_r))
        episode.user_data["agent1_rewards"].append(ag1_r)
        episode.user_data["agent2_rewards"].append(ag2_r)

    def on_episode_end(self, *, worker, base_env,
                       policies, episode,
                       env_index, **kwargs):
        episode.custom_metrics["ag1_R"] = sum(episode.user_data["agent1_rewards"])
        episode.custom_metrics["ag2_R"] = sum(episode.user_data["agent2_rewards"])
        episode.hist_data["agent1_rewards"] = episode.user_data["agent1_rewards"]
        episode.hist_data["agent2_rewards"] = episode.user_data["agent2_rewards"]



In [ ]:
# Setting up our config to point to our new custom callbacks class:
config.update({
    "env": MultiAgentArena,  # force "reload"
    "callbacks": MyCallbacks,  # by default, this would point to `rllib.agents.callbacks.DefaultCallbacks`, which does nothing.
    # Revert these to single trials.
    "lr": 0.0001,
    "train_batch_size": 4000,
})

tune.run("PPO", config=config, stop={"training_iteration": 10})

In [ ]:
# Exercise #3:
# ============
# The episode mean rewards reported to us thus far were always the sum
# of both agents, which doesn't seem to make too much sense given that
# the agents are adversarial.
# Instead, we would like to know, what the individual agents' rewards are in
# our environment.
# Write your own custom callback class (sub-class
# ray.rllib.agents.callback::DefaultCallbacks) and override one or more methods
# therein to manipulate and collect the following data:

#TODO

# a) Extract each agent's individual rewards from ...
# b) Store each agents reward under the new "reward_agent1" and
#    "reward_agent2" keys in the custom metrics.
# c) Run a simple experiment and confirm that you are seeing these two new stats
#    in the tensorboard output.
# Good luck! :)

